<a href="https://colab.research.google.com/github/mlaricobar/Machine-Learning-Course/blob/master/%5B4%5D_ALS_Implicit_Collaborative_Filtering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Filtro Colaborativo Implícito con ALS**

<img src="https://cdn-images-1.medium.com/max/2400/1*6y8nVYtQef_jW-ZeKY2zMw.png" />

Papers de Referencia:

1.   Collaborative Filtering for Implicit Feedback Datasets: http://yifanhu.net/PUB/cf.pdf
2.   





## Contenido:


1.   Introducción
2.   Implícito vs Explícito
3.   El dataset
4.   Alternating least squares
5.   Ítems similares
6.   Hacer recomendaciones

## 1. Introducción


En este notebook voy a implementar un algoritmo de recomendación implícito. Buscaré ser capaz de encontrar ítems similares y dar recomendaciones a los usuarios. En este notebook cubriremos la parte teórica, matemática así como algunas implementaciones en python.

Debido a que nos enfocaremos en la técnica del filtro colaborativo, solo nos enfocaremos en los ítems, usuarios y en los ítems con los que los usuarios han interactuado.

## 2. Datos Explícitos vs Implícitos

### 2.1. Datos Explícitos

Son aquellos conjuntos de datos en los que se cuenta con un concepto de rating que hace referencia a la valoración que le da un usuario a un producto en específico, por ejemplo en los datasets de Netflix o MovieLens encontraremos ratings que se encuentran entre 1 a 5. Con este dato podemos saber que tanto a un usuario le gusta o no un ítem lo que es genial, sin embargo es difícil obtener estos datos. Esto es debido a que los usuarios por lo general no utilizan estas opciones de valoración de productos.

### 2.2. Datos Implícitos

Son aquellos datos que extraemos del comportamiento de los usuarios, sin ratings ni acciones específicas necesarias. Por ejemplo podrían ser los ítems que un usuario ha comprado, cuántas veces los usuarios han reproducido una canción o visto una película, cuánto tiempo ha pasado un usuario leyendo un artñiculo, etc. La ventaja es que tenemos muchos más datos, sin embargo la desventaje es que son más ruidosos y no siempre es evidente lo que significan.

Por ejemplo, con los ratings de estrellas sabemos que 1 significa que a un usuario no le gusta un ítem y 5 que realmente le encanta. Con la reproducción de canciones puede ser que un usuario haya reproducido una canción y la haya odiado, o le haya encantado, o en algún lugar intermedio, y si no reproduciera una canción, es posible que no les guste o que le encantaría conocer para reproducirla.

En esta oportunidad nos centraremos en lo que sabemos que el usuario ha consumido y en la confianza que tenemos en si les gusta o no algún ítem en particular. **Por ejemplo, podemos medir la frecuencia con la que reproducen una canción y asumir una mayor confianza si la han escuchado 500 veces en comparación con una vez.**

Las recomendaciones implícitas se están convirtiendo en un aspecto cada vez más importante de muchos sistemas de recomendación a medida que crece la cantidad de datos implícitos. Por ejemplo, el desafío original de Netflix se centró solo en datos explícitos, pero ahora están confiando cada vez más en señales implícitas. Lo mismo ocurre con Hulu, Spotify, Etsy y muchos otros.

## 3. El dataset

En esta oportunidad se utilizará el dataset de **Lastfm** que contiene el comportamiento de escucha de 360,000 usuarios.

Además, contiene la identificación del usuario, la identificación del artista, el nombre de los artistas y el número de veces que un usuario escuchó a un artista determinado. La descarga también contiene un archivo con edades de usuario, el género y su nacionalidad, etc., pero no lo usaremos por el momento.

## 4. Alternating Least Squares

Alternating Least Squares (ALS) es el modelo que usaremos para ajustar nuestros datos y encontrar similitudes. Pero antes de enfocarnos en cómo funciona, deberíamos analizar algunos de los conceptos básicos de la factorización matricial, que es lo que pretendemos utilizar en el ALS.

### 4.1. Factorización de Matrices

La idea es básicamente tomar una matriz grande (o potencialmente enorme) y factorizarla en una representación más pequeña de la matriz original. Puede pensarlo de la misma manera que tomaríamos un gran número y lo dividiríamos en dos números primos mucho más pequeños. Terminamos con dos o más matrices de dimensiones inferiores cuyo producto es igual al original.

Cuando hablamos de filtrado colaborativo para sistemas de recomendación, queremos resolver el problema de nuestra matriz original con millones de dimensiones diferentes, pero nuestros "gustos" no son tan complejos. Incluso si he visto cientos de artículos, podrían expresar un par de gustos diferentes. Aquí podemos utilizar realmente la factorización de matriz para reducir matemáticamente la dimensionalidad de nuestra matriz original de "todos los usuarios por todos los elementos" en algo mucho más pequeño que represente "todos los elementos por algunas dimensiones de gusto" y "todos los usuarios por algunas dimensiones de gusto". Estas dimensiones se denominan características latentes u ocultas y las aprendemos de nuestros datos.

Hacer esta reducción y trabajar con menos dimensiones hace que sea mucho más eficiente desde el punto de vista informático y, a la vez, nos proporciona mejores resultados, ya que podemos razonar acerca de los elementos en este "espacio del gusto" más compacto.

**Si podemos expresar a cada usuario como un vector de sus valores de gusto, y al mismo tiempo expresar cada elemento como un vector de los gustos que representan. Puedes ver que podemos hacer una recomendación muy fácilmente**. Esto también nos da la posibilidad de encontrar conexiones entre usuarios que no tienen elementos específicos en común pero que comparten gustos comunes.

Ahora debe notarse que no tenemos idea de cuáles son realmente estas características o gustos. No podremos etiquetarlos como "rock" o "de ritmo rápido" o "con Jay-Z". No reflejan necesariamente metadatos reales.


### 4.1. Factorización de Matrices en Datos Implícitos

Hay diferentes formas de factorizar una matriz, como la descomposición del valor singular (SVD) o el análisis semántico latente probabilístico (PLSA) si estamos tratando con datos explícitos.

Con los datos implícitos, la diferencia radica en cómo manejamos todos los datos faltantes en nuestra muy escasa matriz. Para datos explícitos, los tratamos como campos desconocidos a los que debemos asignarles una calificación pronosticada. Pero implícitamente no podemos simplemente asumir lo mismo, ya que también hay información en estos valores desconocidos. **Como se indicó anteriormente, no sabemos si un valor faltante significa que al usuario no le gustó algo, o si significa que lo ama pero simplemente no lo sabe. Básicamente necesitamos alguna forma de aprender de los datos faltantes. Así que necesitaremos un enfoque diferente para llevarnos allí.**

#### Volver a ALS

ALS es un proceso de optimización iterativo en el que, en cada iteración, intentamos acercarnos cada vez más a una representación factorizada de nuestros datos originales.

Tenemos nuestra matriz original R de tamaño u x i con nuestros usuarios, artículos y algún tipo de información de retroalimentación (o feedback). Luego queremos encontrar una manera de convertir eso en una matriz con usuarios y características ocultas de tamaño u x f y una con elementos y características ocultas de tamaño f x i. En U y V tenemos pesos sobre cómo se relaciona cada usuario / elemento con cada característica. Lo que hacemos es calcular U y V para que su producto se aproxime a R lo más cerca posible: R ≈ U x V.

<img src="https://cdn-images-1.medium.com/max/2400/1*ygHEXIhg5FtkSD3UQaldgw.png" />

Asignando aleatoriamente los valores en U y V y utilizando mínimos cuadrados de forma iterativa, podemos llegar a qué pesos producen la mejor aproximación de R. El enfoque de mínimos cuadrados en sus formas básicas significa ajustar una línea a los datos, midiendo la suma de las distancias al cuadrado. Todos los puntos apuntan a la línea e intentan obtener un ajuste óptimo minimizando este valor.

Con el enfoque de mínimos cuadrados alternos usamos la misma idea, pero alternamos iterativamente entre la optimización de U y la fijación de V y viceversa. Hacemos esto para que cada iteración se acerque a R = U x V.

El enfoque que vamos a utilizar con nuestro conjunto de datos implícitos es el que se describe en el filtrado colaborativo para conjuntos de datos de retroalimentación implícita de Hu, Korenand y Volinsky (y utilizado por Facebook y Spotify). Su solución es muy sencilla, así que solo voy a explicar la idea general y la implementación, pero definitivamente debería leerla.

Su solución es fusionar la preferencia (p) para un ítem con la confianza (c) que tenemos para esa preferencia. **Comenzamos con los valores faltantes como una preferencia negativa con un valor de confianza bajo y los valores existentes una preferencia positiva pero con un valor de confianza alto.** Podemos usar algo como el conteo de reproducciones, el tiempo dedicado a una página o alguna otra forma de interacción como base para calcular nuestra confianza.



*   Establecemos la preferencia (p):

<img src="https://cdn-images-1.medium.com/max/1600/1*fzBWtw4JtADMA_PjI8fUjA.png" />

Básicamente nuestra preferencia es una representación binaria de nuestros datos de retroalimentación r. Si la respuesta es mayor que cero, la configuramos en 1. Tiene sentido.

*   La confianza (c) se calcula de la siguiente manera:

<img src="https://cdn-images-1.medium.com/max/1600/1*1rnDwWv6upHMQAeoFI_izQ.png" />

Aquí, la confianza se calcula utilizando la magnitud de r (los datos de retroalimentación o feedback), lo que nos da una confianza mayor cuanto más veces un usuario ha reproducido, visto o hecho clic en un elemento. La tasa a la que aumenta nuestra confianza se establece a través de un factor de escala lineal α. También agregamos 1, por lo que tenemos una confianza mínima incluso si α x r es igual a cero.

Esto también significa que incluso si solo tenemos una interacción entre un usuario y un elemento, la confianza será mayor que la de los datos desconocidos dado el valor α. **En el documento encontraron que α = 40 funcionaba bien y en algún lugar entre 15 y 40 funcionaron para mí.**

El objetivo ahora es encontrar el vector para cada usuario (xu) y el elemento (yi) en las dimensiones de la función, lo que significa que queremos minimizar la siguiente función de pérdida:

<img src="https://cdn-images-1.medium.com/max/1600/1*i6-tSA97cOAGVldVUsiKHw.png" />

Como señala el paper, si solucionamos los factores de usuario o los factores de elemento, podemos calcular un mínimo global. La derivada de la ecuación anterior nos brinda la siguiente ecuación para minimizar la pérdida de nuestros usuarios:

<img src="https://cdn-images-1.medium.com/max/1600/1*YKevKqOWHKGiKNGMDT8Rnw.png">

Y esto para minimizarlo para nuestros artículos:

<img src="https://cdn-images-1.medium.com/max/1600/1*mbiRya9U7A91Q32jMWNQwA.png">

Un paso más es que al darse cuenta de que el producto de la transpuesta de Y, Cu e Y se pueden desglosar como se muestra a continuación:

<img src="https://cdn-images-1.medium.com/max/1600/1*Spm4pEFTMisdAv5CNGOjfA.png">

Ahora tenemos Y-transpuesta.Y y X-transpuesta.X independientes de u e i, lo que significa que podemos calcularlo y hacer que el cálculo sea mucho menos intensivo. Así que con eso en mente, nuestras ecuacionaes finales para el usuario e ítem son:

<img src="https://cdn-images-1.medium.com/max/1600/1*Ppl1LxSu3zJsLXtck8NMRQ.jpeg">


*   X e Y: Nuestras matrices de usuarios y elementos inicializados aleatoriamente. Estos se actualizarán alternativamente.
*   Cu y Ci: Nuestros valores de confianza.
*   λ: Regularizador para reducir el sobreajuste (estamos usando 0.1).
*   p (u) y p (i): la preferencia binaria para un elemento. Una si conocemos la preferencia y cero si no la conocemos.
*   I: la matriz de identidad. Matriz cuadrada con unos en diagonal y ceros en cualquier otra parte.

Al iterar entre las dos ecuaciones anteriores, llegamos a una matriz con vectores de usuario y otra con vectores de elementos que luego podemos usar para generar recomendaciones o encontrar similitudes.


## Artículos similares


Para calcular la similitud entre los elementos, calculamos el punto-producto entre nuestros vectores de elementos y su transposición. Entonces, si queremos que artistas similares digan Joy Division, tomamos el producto punto entre todos los vectores de elementos y la transposición del vector de elementos de Joy Division. Esto nos dará la puntuación de similitud:

<img src="https://cdn-images-1.medium.com/max/1600/1*aeQO6uL29wzQy8C1pz6Wwg.png">

## Haciendo recomendaciones


Para hacer recomendaciones para un usuario determinado, tomamos un enfoque similar. Aquí calculamos el producto de punto entre nuestro vector de usuario y la transposición de nuestros vectores de elementos. Esto nos da una puntuación de recomendación para nuestro usuario y cada elemento:

<img src="https://cdn-images-1.medium.com/max/1600/1*UGxdsKnvkekwW0IQNrj1ig.png">

## Ok, vamos a escribirlo!


Primero importaremos las librerías que necesitamos y cargaremos el conjunto de datos de lastfm. También tendremos que hacer algunos arreglos para que los datos tomen la forma que queremos.

In [0]:
raw_data.head()

In [0]:
raw_data.info(null_counts=True)

Luego creamos nuestra matriz dispersa R de tamaño usuarios x ítems.  El uso de una matriz dispersa nos permite almacenar solo los valores que están realmente allí y no todos los que faltan (que es aproximadamente el 99% del conjunto de datos).

Ahora que tenemos nuestros datos preparados y listos para comenzar, podemos comenzar con una primera implementación de nuestra función ALS implícita.

Comenzamos calculando la confianza para todos los usuarios y elementos, creamos nuestras matrices X e Y para mantener nuestros vectores de usuario y elemento y asignamos los valores al azar. También precomputamos nuestras diagonales.

In [0]:
def implicit_als(sparse_data, alpha_val=40, iterations=10, lambda_val=0.1, features=10):
  
  """ 
  Implementación del Alternating Least Squares con datos implícitos. De forma iterativa se
  calcula los vectores de usuario (x_u) e ítems (y_i) usando las siguientes formulas:
 
    x_u = ((Y.T*Y + Y.T*(Cu - I) * Y) + lambda*I)^-1 * (X.T * Cu * p(u))
    y_i = ((X.T*X + X.T*(Ci - I) * X) + lambda*I)^-1 * (Y.T * Ci * p(i))
 
  Argumentos:
      sparse_data (csr_matrix): Nuestra matriz dispersa usuario-por-ítem

      alpha_val (int): El ratio en el que incrementaremos nuestra confianza de preferencia
      con más interacciones.

      iterations (int): Cuántas veces alternaremos entre corregir y actualizar los vectores
      de usuarios e ítems.

      lambda_val (float): Valor de regularización.

      features (int): Cuántas características latentes queremos calcular.
  
  Retorna:
      X (csr_matrix): Vector de usuarios del tamaño usuarios-por-características

      Y (csr_matrix): Vector de ítems del tamaño ítems-por-características
  """
  
  # Calcular la confianza por cada valor en nuestro dataset
  confidence = sparse_data * alpha_val

  # Obtener el tamaño de los vectores de usuarios e ítems
  user_size, item_size = sparse_data.shape

  # Creamos nuestro vector de usuarios X del tamaño usuarios-por-características,
  # el vector de ítmes Y del tamaño ítems-por-características y les asignamos valores aleatorios
  X = sparse.csr_matrix(np.random.normal(size = (user_size, features)))
  Y = sparse.csr_matrix(np.random.normal(size = (item_size, features)))

  # Precalculamos I y lambda * I
  X_I = sparse.eye(user_size)
  Y_I = sparse.eye(item_size)

  I = sparse.eye(features)
  lI = lambda_val * I

  """ Continuación de la función implicit_als"""

  # Iniciamos el bucle principal. Por cada iteración calculamos X y luego Y
  for i in range(iterations):
    print('Iteración %d de %d' % (i+1, iterations))

    # Precalculamos Y-transpuesta-Y e X-transpuesta-X
    yTy = Y.T.dot(Y)
    xTx = X.T.dot(X) 

    # Iterar por todos los usuarios
    for u in range(user_size):
      # Obtener el registro de Usuario
      u_row = confidence[u,:].toarray()

      # Calculamos la preferencia binaria p(u)
      p_u = u_row.copy()
      p_u[p_u != 0] = 1.0

      # Calculamos Cu y Cu + I
      CuI = sparse.diags(u_row, [0])
      Cu = CuI + Y_I

      # Put it all together and compute the final formula
      yT_CuI_y = Y.T.dot(CuI).dot(Y)
      yT_Cu_pu = Y.T.dot(Cu).dot(p_u.T)
      X[u] = spsolve(yTy + yT_CuI_y + lI, yT_Cu_pu)

    for i in xrange(item_size):

      # Obtener la columna ítem y la transpuesta.
      i_row = confidence[:,i].T.toarray()

      # Calcular la preferencia binaria p(i)
      p_i = i_row.copy()
      p_i[p_i != 0] = 1.0

      # Calcular Ci y Ci - I
      CiI = sparse.diags(i_row, [0])
      Ci = CiI + X_I

      # Ponlo todo junto y calcula la fórmula final.
      xT_CiI_x = X.T.dot(CiI).dot(X)
      xT_Ci_pi = X.T.dot(Ci).dot(p_i.T)
      Y[i] = spsolve(xTx + xT_CiI_x + lI, xT_Ci_pi)

    return X, Y

Luego llamamos a nuestra función para obtener nuestros vectores de usuario y vectores de ítems. Como verás, si intentas ejecutar esto con el conjunto de datos que estamos usando, te llevará mucho tiempo entrenar. Volveremos a cómo podemos acelerarlo más tarde, pero por ahora podemos intentarlo con una sola iteración en lugar de 20. O podemos dividir los datos sin procesar en algo más manejable, por ejemplo, solo las primeras 100.000 filas.

In [0]:
#user_vecs, item_vecs = implicit_als(data_sparse, iterations=2, features=20, alpha_val=40)

Entonces, cuando tengamos nuestro modelo entrenado, podemos comenzar a hacer algunas recomendaciones. Primero vamos a comenzar por encontrar algunos artistas similares a Jay-Z. Obtenemos la similitud al hablar del producto punto de nuestros vectores de elementos con el vector de elementos del artista.

In [0]:
#------------------------------
# FIND SIMILAR ITEMS
#------------------------------

# Let's find similar artists to Jay-Z. 
# Note that this ID might be different for you if you're using
# the full dataset or if you've sliced it somehow. 
item_id = 10277

# Get the item row for Jay-Z
item_vec = item_vecs[item_id].T

# Calculate the similarity score between Mr Carter and other artists
# and select the top 10 most similar.
scores = item_vecs.dot(item_vec).toarray().reshape(1,-1)[0]
top_10 = np.argsort(scores)[::-1][:10]

artists = []
artist_scores = []

# Get and print the actual artists names and scores
for idx in top_10:
    artists.append(item_lookup.artist.loc[item_lookup.artist_id == str(idx)].iloc[0])
    artist_scores.append(scores[idx])

similar = pd.DataFrame({'artist': artists, 'score': artist_scores})

print(similar)

In [0]:
# Let's say we want to recommend artists for user with ID 2023
user_id = 2023

#------------------------------
# GET ITEMS CONSUMED BY USER
#------------------------------

# Let's print out what the user has listened to
consumed_idx = data_sparse[user_id,:].nonzero()[1].astype(str)
consumed_items = item_lookup.loc[item_lookup.artist_id.isin(consumed_idx)]
print consumed_items


#------------------------------
# CREATE USER RECOMMENDATIONS
#------------------------------

def recommend(user_id, data_sparse, user_vecs, item_vecs, item_lookup, num_items=10):
    """Recommend items for a given user given a trained model
    
    Args:
        user_id (int): The id of the user we want to create recommendations for.
        
        data_sparse (csr_matrix): Our original training data.
        
        user_vecs (csr_matrix): The trained user x features vectors
        
        item_vecs (csr_matrix): The trained item x features vectors
        
        item_lookup (pandas.DataFrame): Used to map artist ids to artist names
        
        num_items (int): How many recommendations we want to return:
        
    Returns:
        recommendations (pandas.DataFrame): DataFrame with num_items artist names and scores
    
    """
  
    # Get all interactions by the user
    user_interactions = data_sparse[user_id,:].toarray()

    # We don't want to recommend items the user has consumed. So let's
    # set them all to 0 and the unknowns to 1.
    user_interactions = user_interactions.reshape(-1) + 1 #Reshape to turn into 1D array
    user_interactions[user_interactions > 1] = 0

    # This is where we calculate the recommendation by taking the 
    # dot-product of the user vectors with the item vectors.
    rec_vector = user_vecs[user_id,:].dot(item_vecs.T).toarray()

    # Let's scale our scores between 0 and 1 to make it all easier to interpret.
    min_max = MinMaxScaler()
    rec_vector_scaled = min_max.fit_transform(rec_vector.reshape(-1,1))[:,0]
    recommend_vector = user_interactions*rec_vector_scaled
   
    # Get all the artist indices in order of recommendations (descending) and
    # select only the top "num_items" items. 
    item_idx = np.argsort(recommend_vector)[::-1][:num_items]

    artists = []
    scores = []

    # Loop through our recommended artist indicies and look up the actial artist name
    for idx in item_idx:
        artists.append(item_lookup.artist.loc[item_lookup.artist_id == str(idx)].iloc[0])
        scores.append(recommend_vector[idx])

    # Create a new dataframe with recommended artist names and scores
    recommendations = pd.DataFrame({'artist': artists, 'score': scores})
    
    return recommendations

# Let's generate and print our recommendations
recommendations = recommend(user_id, data_sparse, user_vecs, item_vecs, item_lookup)
print(recommendations)

In [0]:
def nonzeros(m, row):
  for index in range(m.indptr[row], m.indptr[row+1]):
    yield m.indices[index], m.data[index]
      
      
def implicit_als_cg(Cui, features=20, iterations=20, lambda_val=0.1):
  user_size, item_size = Cui.shape
  
  X = np.random.rand(user_size, features) * 0.01
  Y = np.random.rand(item_size, features) * 0.01
  
  Cui, Ciu = Cui.tocsr(), Cui.T.tocsr()

  for iteration in range(iterations):
      print ('iteration %d of %d' % (iteration+1, iterations))
      least_squares_cg(Cui, X, Y, lambda_val)
      least_squares_cg(Ciu, Y, X, lambda_val)
    
  return sparse.csr_matrix(X), sparse.csr_matrix(Y)
  
  
def least_squares_cg(Cui, X, Y, lambda_val, cg_steps=3):
  users, features = X.shape

  YtY = Y.T.dot(Y) + lambda_val * np.eye(features)

  for u in range(users):

      x = X[u]
      r = -YtY.dot(x)

      for i, confidence in nonzeros(Cui, u):
          r += (confidence - (confidence - 1) * Y[i].dot(x)) * Y[i]

      p = r.copy()
      rsold = r.dot(r)

      for it in range(cg_steps):
          Ap = YtY.dot(p)
          for i, confidence in nonzeros(Cui, u):
              Ap += (confidence - 1) * Y[i].dot(p) * Y[i]

          alpha = rsold / p.dot(Ap)
          x += alpha * p
          r -= alpha * Ap

          rsnew = r.dot(r)
          p = r + (rsnew / rsold) * p
          rsold = rsnew

      X[u] = x

alpha_val = 15
conf_data = (data_sparse * alpha_val).astype('double')
user_vecs, item_vecs = implicit_als_cg(conf_data, iterations=20, features=20)

# Código de Filtrado Colaborativo con Tipo de Entrada Implícita

In [0]:
!pip install implicit

In [0]:
import sys
import random
import implicit
import numpy as np
import pandas as pd
import scipy.sparse as sparse

from google.colab import drive
from scipy.sparse.linalg import spsolve
from sklearn.preprocessing import MinMaxScaler

In [0]:
drive.mount('/gdrive')

In [0]:
!ls -l /gdrive/'My Drive'/Tesis/Blogs/Data/Last.fm

# Creación del Sistema de Recomendación con datos Implícitos

In [0]:
# Load the data like we did before
raw_data = pd.read_table('/gdrive/My Drive/Tesis/Blogs/Data/Last.fm/lastfm-dataset-360K/usersha1-artmbid-artname-plays.tsv', usecols = [0, 2, 3], header=None,  names=['user', 'artist', 'plays'])

In [0]:
# Drop NaN columns
data = raw_data.dropna()
data = data.copy()

# Create a numeric user_id and artist_id column
data['user'] = data['user'].astype("category")
data['artist'] = data['artist'].astype("category")
data['user_id'] = data['user'].cat.codes
data['artist_id'] = data['artist'].cat.codes

In [0]:
# The implicit library expects data as a item-user matrix so we
# create two matricies, one for fitting the model (item-user) 
# and one for recommendations (user-item)
sparse_item_user = sparse.csr_matrix((data['plays'].astype(float), (data['artist_id'], data['user_id'])))
sparse_user_item = sparse.csr_matrix((data['plays'].astype(float), (data['user_id'], data['artist_id'])))

# Initialize the als model and fit it using the sparse item-user matrix
model = implicit.als.AlternatingLeastSquares(factors=20, regularization=0.1, iterations=20)

# Calculate the confidence by multiplying it by our alpha value.
alpha_val = 15
data_conf = (sparse_item_user * alpha_val).astype('double')

#Fit the model
model.fit(data_conf)

## Identifica a tu Artista

In [0]:
data.loc[data["artist"].apply(lambda a: "nirvana" in a.lower()), :].groupby(["artist", "artist_id"], as_index=False).agg({"user": "count"}).sort_values(by="user", ascending=False).rename(columns={"user": "q_users"})

## Encuentra a los Similares

In [0]:
#---------------------
# FIND SIMILAR ITEMS
#---------------------

# Find the 10 most similar to Jay-Z
item_id = 199842 #Nirvana
n_similar = 30

# Use implicit to get similar items.
similar = model.similar_items(item_id, n_similar)

# Print the names of our most similar artists
for item in similar:
    idx, score = item
    print (data.artist.loc[data.artist_id == idx].iloc[0])

## Recomendaciones de Usuarios

In [0]:
#------------------------------
# CREATE USER RECOMMENDATIONS
#------------------------------

# Create recommendations for user with id 2025
user_id = 198823

# Use the implicit recommender.
recommended = model.recommend(user_id, sparse_user_item)

#### ¿Que vió anteriormente?

In [0]:
data.loc[data["user_id"] == 198823]

#### Sus recomendaciones

In [0]:
artists = []
scores = []

# Get artist names from ids
for item in recommended:
    idx, score = item
    artists.append(data.artist.loc[data.artist_id == idx].iloc[0])
    scores.append(score)

# Create a dataframe of artist names and scores
recommendations = pd.DataFrame({'artist': artists, 'score': scores})

print (recommendations)